In [1]:
import numpy as np
import pandas


from sklearn import linear_model 

# from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import train_test_split

#'Linear model for testing the individual effect of each of many regressors. '
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn import preprocessing

from sklearn.cross_validation import KFold
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import PolynomialFeatures

from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score

/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#import data
FB_data = pandas.read_csv('facebook_train.csv')

FB_headers = FB_data.columns.values.tolist()[:-3]
FB_data.head()

,total_followers,type,category,month,weekday,hour,paid,comment,like,share
0,124050,Photo,3,6,4,17,1,3,77,16
1,132556,Photo,2,8,3,24,0,11,313,50
2,92786,Photo,2,2,6,17,1,2,145,39
3,135632,Photo,1,9,2,1,0,2,299,49
4,139540,Link,1,12,3,17,1,7,229,21


In [17]:
# orginize data

encode_data = DataFrameMapper([
    ('total_followers', preprocessing.StandardScaler()),
    ('type', preprocessing.LabelBinarizer()),
    (['category'],preprocessing.OneHotEncoder()),
    (['month'], preprocessing.OneHotEncoder()),
    (['weekday'],  preprocessing.OneHotEncoder()),
    (['hour'], None),
    (['paid'], None), 
    (['share'], None),
    (['like'], None),
    (['comment'], None) ], df_out= True)


FB_DATA = encode_data.fit_transform(FB_data)
FB_DATA.head()

/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


,total_followers,type_Link,type_Photo,type_Status,type_Video,category_0,category_1,category_2,month_0,month_1,...,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,hour,paid,share,like,comment
0,0.036279,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,17.0,1.0,16.0,77.0,3.0
1,0.555526,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,24.0,0.0,50.0,313.0,11.0
2,-1.872224,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,17.0,1.0,39.0,145.0,2.0
3,0.743300,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,49.0,299.0,2.0
4,0.981863,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,17.0,1.0,21.0,229.0,7.0


In [18]:
# seperate x to y
FB_m_headers = FB_DATA.columns.values.tolist()[:-3]
X_FB = FB_DATA[FB_m_headers]
FB_l = FB_DATA.like
FB_c = FB_DATA.comment
FB_s = FB_DATA.share

FB_l_log = np.log(FB_l + 1)

X_FB.shape

(397, 29)

In [76]:
# encode_data.transformed_names_

In [115]:
#opt 1 :pipeline - polynom +feature selection in CV , using linear regression

def reg_pipe (X, Y):
    (x_train, x_test, y_train, y_test) = train_test_split(X, Y, test_size=0.1, random_state=5)

    polynomial_features = PolynomialFeatures(include_bias=False)
    linear_regression = linear_model.LinearRegression()

    pipeline1 = Pipeline([ ("polynomial_features", polynomial_features),
                           ('select', SelectKBest(f_regression)),
                           ("linear_regression", linear_regression)])

    total_scores = []

    for i in range(1,3):
        scores = []
        for K in range(1,15):

    #     print pipeline1.get_params()
            pipeline1.set_params(polynomial_features__degree = i, select__k=K)
            pipeline1.fit(x_train, y_train)

            # Evaluate the models using crossvalidation
            scores.append(cross_val_score(pipeline1, x_test, y_test, scoring="r2", cv=7))

    #         
            total_scores.append(np.mean(scores))
    print
    print np.mean(total_scores)



In [117]:
log_c = np.log(FB_c+1)
norm_c = (log_c - np.mean(log_c))/np.std(log_c)
reg_pipe(X_FB, norm_c)



pipeline1.set_params(select__k=11)
pipeline1.fit(x_train, y_train)
test_predictions = pipeline1.predict(x_test)

for metric in (metrics.mean_squared_error, metrics.mean_absolute_error, 
               metrics.median_absolute_error, metrics.r2_score): 
    
    print(metric.__name__, metric(test_predictions, y_test))

-0.380904008934


In [123]:
#opt 2: feature selection and cv using linear regression
def regressor (X, Y):
    
    def print_err(x_to_train, x_to_test, y_train, y_test, regr):
        
        regr.fit(x_to_train, y_train)
        y_predict = regr.predict(x_to_test)
        
        print 'regression co {}'.format(regr.coef_)
        print 
    
        print 'MSE : {}'.format(metrics.mean_squared_error(y_test, y_predict))
        print 'r^2 : {}'.format(metrics.r2_score(y_predict, y_test))

        for metric in (metrics.mean_squared_error, metrics.mean_absolute_error, 
               metrics.median_absolute_error, metrics.r2_score): 
    
            print(metric.__name__, metric(y_predict, y_test))
    
    
    regr = linear_model.LinearRegression()

    (x_train, x_test, y_train, y_test) = train_test_split(X, Y, test_size=0.1, random_state=5)
    
    (x_train, x_test, y_train, y_test) = np.array(x_train), np.array(x_test), np.array(y_train), np.array(y_test)   
    
    scores = {}
    mean_scores = {}
    features = {}
    
    # iterates on num of features
    for K in range(3, 15):
        
        scores[K] = []
        mean_scores[K] = []
        features[K]= []


        #K attributs in X that fit y best (based on F-value between label/feature for regression tasks.)
        select_features = SelectKBest(f_regression, k=K)
        select_features.fit(x_train, y_train)


        # takes the indexes of values chosen above 
#         print x_train
        x_train_f = np.array(x_train)[:, select_features.get_support()]
        x_test_f = np.array(x_test)[:, select_features.get_support()]

        # fit the model with the new data sets
        regr.fit(x_train_f, y_train)
        err = cross_val_score(regr, x_test_f, y_test, scoring="mean_squared_error", cv=10)


        # store the error
        scores[K].append(err)

        features[K] = select_features.get_support()

        mean_scores[K] = (np.average(scores[K]))
        


    # print scores
#     print mean_scores
    
    opt_k  = min(mean_scores, key=mean_scores.get) 
    opt_k_values = features[opt_k]
    
    print np.array(encode_data.transformed_names_)[opt_k_values]

    
    x_to_train = x_train[:, features[opt_k]]
    x_to_test = x_test[:, features[opt_k]]
    
    print 'optimal num of features: {}'.format(opt_k)

    
    return print_err(x_to_train, x_to_test, y_train, y_test ,regr)

    





In [124]:
### comment
log_c = np.log(FB_c+1)
norm_c = (log_c - np.mean(log_c))/np.std(log_c)

regressor(X_FB, FB_c)


/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sam

['type_Link' 'category_0' 'category_1' 'month_1' 'month_2' 'month_6'
 'month_8' 'month_9' 'weekday_0' 'weekday_2' 'weekday_3' 'weekday_6' 'paid']
optimal num of features: 13
regression co [ -3.20481525  -0.27291544   5.41563964   7.16613974  -2.52785611
  10.5567096    4.5326534   -0.36891759   4.29341986  -1.40165857
  -3.15735283   9.16818315   2.75664655]

MSE : 220.700273195
r^2 : -3.66886766636
('mean_squared_error', 220.70027319460286)
('mean_absolute_error', 10.384115238856035)
('median_absolute_error', 7.9650057287873643)
('r2_score', -3.6688676663634787)


/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sam

In [125]:
### likes

log_l = np.log(FB_l+1)
norm_l = (log_l - np.mean(log_l))/np.std(log_l)
regressor(X_FB, FB_l)




/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sam

['type_Link' 'category_0' 'category_1' 'category_2' 'month_2' 'month_5'
 'month_6' 'month_8' 'weekday_2' 'weekday_3' 'weekday_6' 'paid']
optimal num of features: 12
regression co [ -1.16147892e+02   2.30478736e+16   2.30478736e+16   2.30478736e+16
  -7.09208330e+01  -1.11500602e+02   8.85423336e+01   5.50862396e+01
  -5.10563154e+01  -6.27608331e+01   5.44835974e+01   1.10595779e+02]

MSE : 66706.175
r^2 : -5.4073768303
('mean_squared_error', 66706.175000000003)
('mean_absolute_error', 158.72499999999999)
('median_absolute_error', 110.0)
('r2_score', -5.4073768303037992)


/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sam

In [126]:
### shares 

regressor(X_FB, FB_s)


/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sam

['type_Link' 'type_Video' 'category_0' 'category_1' 'category_2' 'month_1'
 'month_2' 'month_5' 'month_6' 'month_7' 'month_8' 'weekday_3' 'weekday_6'
 'paid']
optimal num of features: 14
regression co [ -1.27657233e-01   4.62476375e+01  -3.52234126e+15  -3.52234126e+15
  -3.52234126e+15   1.38327591e+01  -8.57869310e+00  -1.60223657e+01
   1.21793419e+01  -1.90026436e+01   8.00435430e+00  -4.20444064e+00
   1.73760456e+01   9.62521251e+00]

MSE : 949.79375
r^2 : -1.95933295392
('mean_squared_error', 949.79375000000005)
('mean_absolute_error', 22.8125)
('median_absolute_error', 16.5)
('r2_score', -1.9593329539241826)


/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/gilikarni/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sam

http://nbviewer.jupyter.org/github/cs109/content/blob/master/lec_10_cross_val.ipynb

In [118]:
FB_test = pandas.read_csv('facebook_train.csv')
